In [18]:
import pandas as pd
import numpy as np
import re
import pickle
import scipy
import seaborn as sns

In [19]:
slates = pd.read_json("viewpoints.json")
slates_ids = list(slates.columns)
slates_ids

['1gi2awo',
 '1gi378c',
 '1gi5k6b',
 '1gi6rmt',
 '1giwkq5',
 '1giy3mj',
 '1gj6zkq',
 '1gjc9pm',
 '1gjquni',
 '1gjvwc2',
 '1gjy9l8',
 '1gjyyhu',
 '1gk6tzm',
 '1gkbmkz',
 '1gkhfq7',
 '1gkhp7b',
 '1gkukkp']

In [20]:
slates['1gi2awo'].dropna().values

array(['Boyfriend crossed a line by using a personal and sensitive topic to win a game.',
       'He should have known better than to use a dead mom joke, even in a game meant to be offensive.',
       "The game's goal is to be controversial, but that doesn't mean it's okay to hurt someone's feelings.",
       "Boyfriend's apology was insincere, as he implied the OP was overreacting.",
       "OP should set boundaries and communicate what is and isn't okay to joke about.",
       "Boyfriend's actions show he's willing to hurt OP for personal gain, which is a red flag in the relationship.",
       "OP should consider whether boyfriend's behavior is a consistent pattern and if it's a sign of deeper issues.",
       "The game is meant to be uncomfortable, but that doesn't mean OP should just accept being hurt.",
       "Boyfriend's actions were not just about winning the game, but also about targeting OP specifically.",
       "OP's feelings are valid, and boyfriend should have been more 

In [21]:
len(slates.get('1gi2awo').dropna())

10

In [22]:
checkpoints = pd.read_csv("clean_data_pref_checkpoint.csv")
checkpoints["response"].values

array([nan,
       "ORDER = [2, 3, 4, 9, 1, 10, 6, 7, 5, 8]\n\nREASONING: Based on my comment, I agree that going behind someone's back to do something they explicitly said no to is not okay (2). I also think that forcing someone to accept something they don't want is violating their boundaries (3). Sneaking money into someone's wallet is not a good way to show appreciation (4), and not respecting someone's wishes can make them feel uncomfortable or even stressed (9). I agree that a simple 'thank you' or acknowledgement can be enough, without needing to force payment or compensation (10). I agree that everyone involved is trying to do the right thing, but some methods are better than others (6), and it's better to be transparent and direct when showing appreciation, rather than sneaky or manipulative (7). Instead of paying for services, consider doing something nice for the person, like taking them out for a meal (5) and making a charitable donation in someone's honor instead of forcin

In [23]:
filtered_checkpoints = checkpoints[checkpoints['response'].notna() & (checkpoints['response'] != "")]

In [24]:
filtered_checkpoints["Post_ID"].unique() 

array(['1gkukkp', '1gkhp7b', '1gkhfq7', '1gkbmkz', '1gk6tzm', '1gjy9l8',
       '1gjyyhu', '1gjvwc2', '1gjquni', '1gjc9pm', '1gj6zkq', '1giwkq5',
       '1giy3mj', '1gi6rmt', '1gi2awo', '1gi5k6b', '1gi378c'],
      dtype=object)

In [25]:
checkpoints_valid = checkpoints.loc[checkpoints["Post_ID"].isin(slates_ids)]

#sanity check
assert checkpoints_valid["Post_ID"].unique().sort() == slates_ids.sort()

checkpoints_valid

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text,response
0,Post,1gkukkp,AITA for slipping money into my boyfriend’s wa...,AddressAntique2983,2024-11-06 09:22:41,[removed],1382,107,https://www.reddit.com/r/AmItheAsshole/comment...,The final verdict is: **Asshole**\n\nJudgement...,NaN,YTA,[removed],NaN
1,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Brilliant_Storm_3271,2024-11-06 09:29:34,Sometimes you have just got to let people give...,1105,0,NaN,NaN,NaN,NaN,Sometimes you have just got to let people give...,"ORDER = [2, 3, 4, 9, 1, 10, 6, 7, 5, 8]\n\nREA..."
2,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,moneywanted,2024-11-06 11:15:43,NAH - everyone in this story is just a good pe...,96,0,NaN,NaN,NAH,NaN,NAH - everyone in this story is just a good pe...,"ORDER = [7, 6, 1, 5, 9, 10, 2, 3, 4, 8]\nREASO..."
3,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,benbever,2024-11-06 09:32:52,He didn’t want payment. You can offer a few ti...,100,0,NaN,NaN,YTA,NaN,He didn’t want payment. You can offer a few ti...,"ORDER = [2, 4, 3, 1, 9, 10, 7, 6, 8, 5]\n\nREA..."
4,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Ok-Nefariousness1911,2024-11-06 09:33:21,YTA. Yours and your dad's intentions were in t...,182,0,NaN,NaN,YTA,NaN,YTA. Yours and your dad's intentions were in t...,"ORDER = [2, 3, 9, 1, 4, 10, 5, 7, 6, 8]\n\nREA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,Comment,1gi378c,AITA for yelling at my neighbour on the street...,TicketyBoo39,2024-11-03 02:08:07,"Not, but if you yell, then you can't put a pot...",2,0,NaN,NaN,NaN,NaN,"Not, but if you yell, then you can't put a pot...","ORDER = [3, 14, 1, 5, 7, 4, 15, 10, 6, 2, 12, ..."
810,Comment,1gi378c,AITA for yelling at my neighbour on the street...,Fungiblefaith,2024-11-03 02:10:08,"Ok, now i am going to start off saying that I ...",2,0,NaN,NaN,NaN,NaN,"Ok, now i am going to start off saying that I ...","ORDER = [11, 1, 5, 7, 4, 14, 15, 10, 6, 2, 3, ..."
811,Comment,1gi378c,AITA for yelling at my neighbour on the street...,KitchenDismal9258,2024-11-03 03:05:31,NTA\n\nIs the neighbour actually allowed to pa...,2,0,NaN,NaN,NTA,NaN,NTA Is the neighbour actually allowed to park ...,"ORDER = [3,10,14,15,7,5,4,1,11,8,6,2,12,13,9,2..."
812,Comment,1gi378c,AITA for yelling at my neighbour on the street...,Altruistic-Truth8743,2024-11-03 03:16:19,I think you should have walked up to him and t...,2,0,NaN,NaN,NaN,NaN,I think you should have walked up to him and t...,"ORDER = [14, 1, 10, 5, 7, 15, 4, 11, 13, 2, 3,..."


In [26]:
# Function to extract ORDER
def extract_order(text):
    order_match = re.search(r"ORDER\s*=\s*\[([^\]]+)\]", text)
    return [int(x.strip()) for x in order_match.group(1).split(',')] if order_match else None

# Function to extract reasoning
def extract_reasoning(text):
    reasoning_match = re.search(r"REASONING\s*[:=]\s*(.+)", text, flags=re.DOTALL)
    return reasoning_match.group(1).strip() if reasoning_match else None

# Use lambda functions to apply the cleaning logic
checkpoints_valid.loc[checkpoints_valid['Type'] == "Comment", 'order'] = checkpoints_valid.loc[checkpoints_valid['Type'] == "Comment", 'response'].apply(lambda x: extract_order(x))
checkpoints_valid.loc[checkpoints_valid['Type'] == "Comment", 'reasoning'] = checkpoints_valid.loc[checkpoints_valid['Type'] == "Comment", 'response'].apply(lambda x: extract_reasoning(x))
checkpoints_valid

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text,response,order,reasoning
0,Post,1gkukkp,AITA for slipping money into my boyfriend’s wa...,AddressAntique2983,2024-11-06 09:22:41,[removed],1382,107,https://www.reddit.com/r/AmItheAsshole/comment...,The final verdict is: **Asshole**\n\nJudgement...,NaN,YTA,[removed],NaN,NaN,NaN
1,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Brilliant_Storm_3271,2024-11-06 09:29:34,Sometimes you have just got to let people give...,1105,0,NaN,NaN,NaN,NaN,Sometimes you have just got to let people give...,"ORDER = [2, 3, 4, 9, 1, 10, 6, 7, 5, 8]\n\nREA...","[2, 3, 4, 9, 1, 10, 6, 7, 5, 8]","Based on my comment, I agree that going behind..."
2,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,moneywanted,2024-11-06 11:15:43,NAH - everyone in this story is just a good pe...,96,0,NaN,NaN,NAH,NaN,NAH - everyone in this story is just a good pe...,"ORDER = [7, 6, 1, 5, 9, 10, 2, 3, 4, 8]\nREASO...","[7, 6, 1, 5, 9, 10, 2, 3, 4, 8]","Based on my comment, I believe that everyone i..."
3,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,benbever,2024-11-06 09:32:52,He didn’t want payment. You can offer a few ti...,100,0,NaN,NaN,YTA,NaN,He didn’t want payment. You can offer a few ti...,"ORDER = [2, 4, 3, 1, 9, 10, 7, 6, 8, 5]\n\nREA...","[2, 4, 3, 1, 9, 10, 7, 6, 8, 5]","Based on my comment, I think the most liked st..."
4,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Ok-Nefariousness1911,2024-11-06 09:33:21,YTA. Yours and your dad's intentions were in t...,182,0,NaN,NaN,YTA,NaN,YTA. Yours and your dad's intentions were in t...,"ORDER = [2, 3, 9, 1, 4, 10, 5, 7, 6, 8]\n\nREA...","[2, 3, 9, 1, 4, 10, 5, 7, 6, 8]","Based on my comment, I think that going behind..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,Comment,1gi378c,AITA for yelling at my neighbour on the street...,TicketyBoo39,2024-11-03 02:08:07,"Not, but if you yell, then you can't put a pot...",2,0,NaN,NaN,NaN,NaN,"Not, but if you yell, then you can't put a pot...","ORDER = [3, 14, 1, 5, 7, 4, 15, 10, 6, 2, 12, ...","[3, 14, 1, 5, 7, 4, 15, 10, 6, 2, 12, 13, 9, 1...","Based on my comment, I do not condone yelling ..."
810,Comment,1gi378c,AITA for yelling at my neighbour on the street...,Fungiblefaith,2024-11-03 02:10:08,"Ok, now i am going to start off saying that I ...",2,0,NaN,NaN,NaN,NaN,"Ok, now i am going to start off saying that I ...","ORDER = [11, 1, 5, 7, 4, 14, 15, 10, 6, 2, 3, ...","[11, 1, 5, 7, 4, 14, 15, 10, 6, 2, 3, 8, 12, 1...","Based on my comment, I am an asshole who would..."
811,Comment,1gi378c,AITA for yelling at my neighbour on the street...,KitchenDismal9258,2024-11-03 03:05:31,NTA\n\nIs the neighbour actually allowed to pa...,2,0,NaN,NaN,NTA,NaN,NTA Is the neighbour actually allowed to park ...,"ORDER = [3,10,14,15,7,5,4,1,11,8,6,2,12,13,9,2...","[3, 10, 14, 15, 7, 5, 4, 1, 11, 8, 6, 2, 12, 1...","Based on my comment, I don't think the OP is a..."
812,Comment,1gi378c,AITA for yelling at my neighbour on the street...,Altruistic-Truth8743,2024-11-03 03:16:19,I think you should have walked up to him and t...,2,0,NaN,NaN,NaN,NaN,I think you should have walked up to him and t...,"ORDER = [14, 1, 10, 5, 7, 15, 4, 11, 13, 2, 3,...","[14, 1, 10, 5, 7, 15, 4, 11, 13, 2, 3, 6, 8, 1...","Based on my comment, I think the best course o..."


In [27]:
# checkpoints_valid.to_csv("checkpoints_valid_preferences.csv", index=False)

In [28]:
checkpoints_valid.loc[checkpoints_valid["Type"] == "Comment"] 

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text,response,order,reasoning
1,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Brilliant_Storm_3271,2024-11-06 09:29:34,Sometimes you have just got to let people give...,1105,0,NaN,NaN,NaN,NaN,Sometimes you have just got to let people give...,"ORDER = [2, 3, 4, 9, 1, 10, 6, 7, 5, 8]\n\nREA...","[2, 3, 4, 9, 1, 10, 6, 7, 5, 8]","Based on my comment, I agree that going behind..."
2,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,moneywanted,2024-11-06 11:15:43,NAH - everyone in this story is just a good pe...,96,0,NaN,NaN,NAH,NaN,NAH - everyone in this story is just a good pe...,"ORDER = [7, 6, 1, 5, 9, 10, 2, 3, 4, 8]\nREASO...","[7, 6, 1, 5, 9, 10, 2, 3, 4, 8]","Based on my comment, I believe that everyone i..."
3,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,benbever,2024-11-06 09:32:52,He didn’t want payment. You can offer a few ti...,100,0,NaN,NaN,YTA,NaN,He didn’t want payment. You can offer a few ti...,"ORDER = [2, 4, 3, 1, 9, 10, 7, 6, 8, 5]\n\nREA...","[2, 4, 3, 1, 9, 10, 7, 6, 8, 5]","Based on my comment, I think the most liked st..."
4,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Ok-Nefariousness1911,2024-11-06 09:33:21,YTA. Yours and your dad's intentions were in t...,182,0,NaN,NaN,YTA,NaN,YTA. Yours and your dad's intentions were in t...,"ORDER = [2, 3, 9, 1, 4, 10, 5, 7, 6, 8]\n\nREA...","[2, 3, 9, 1, 4, 10, 5, 7, 6, 8]","Based on my comment, I think that going behind..."
5,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,andysjs2003,2024-11-06 09:37:20,"All in all these are nice problems to have, bo...",20,0,NaN,NaN,NaN,NaN,"All in all these are nice problems to have, bo...","ORDER = [2, 4, 3, 1, 9, 10, 6, 7, 5, 8]\n\nREA...","[2, 4, 3, 1, 9, 10, 6, 7, 5, 8]","Based on my comment, I believe that going behi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,Comment,1gi378c,AITA for yelling at my neighbour on the street...,TicketyBoo39,2024-11-03 02:08:07,"Not, but if you yell, then you can't put a pot...",2,0,NaN,NaN,NaN,NaN,"Not, but if you yell, then you can't put a pot...","ORDER = [3, 14, 1, 5, 7, 4, 15, 10, 6, 2, 12, ...","[3, 14, 1, 5, 7, 4, 15, 10, 6, 2, 12, 13, 9, 1...","Based on my comment, I do not condone yelling ..."
810,Comment,1gi378c,AITA for yelling at my neighbour on the street...,Fungiblefaith,2024-11-03 02:10:08,"Ok, now i am going to start off saying that I ...",2,0,NaN,NaN,NaN,NaN,"Ok, now i am going to start off saying that I ...","ORDER = [11, 1, 5, 7, 4, 14, 15, 10, 6, 2, 3, ...","[11, 1, 5, 7, 4, 14, 15, 10, 6, 2, 3, 8, 12, 1...","Based on my comment, I am an asshole who would..."
811,Comment,1gi378c,AITA for yelling at my neighbour on the street...,KitchenDismal9258,2024-11-03 03:05:31,NTA\n\nIs the neighbour actually allowed to pa...,2,0,NaN,NaN,NTA,NaN,NTA Is the neighbour actually allowed to park ...,"ORDER = [3,10,14,15,7,5,4,1,11,8,6,2,12,13,9,2...","[3, 10, 14, 15, 7, 5, 4, 1, 11, 8, 6, 2, 12, 1...","Based on my comment, I don't think the OP is a..."
812,Comment,1gi378c,AITA for yelling at my neighbour on the street...,Altruistic-Truth8743,2024-11-03 03:16:19,I think you should have walked up to him and t...,2,0,NaN,NaN,NaN,NaN,I think you should have walked up to him and t...,"ORDER = [14, 1, 10, 5, 7, 15, 4, 11, 13, 2, 3,...","[14, 1, 10, 5, 7, 15, 4, 11, 13, 2, 3, 6, 8, 1...","Based on my comment, I think the best course o..."


In [29]:
comments_checkpoints_valid = checkpoints_valid.loc[checkpoints_valid["Type"] == "Comment"]

def validate_order_length(row):
    try:
        # Check if the length of the 'order' matches the expected length from the 'slates' dataframe
        expected_len = len(slates.get(row["Post_ID"]).dropna()) if slates.get(row["Post_ID"]) is not None else None
        return len(row["order"]) == expected_len if expected_len is not None else False
    except Exception as e:
        # If an error occurs, print the row details and expected length
        print(f"Error processing row: {row['Post_ID']}")
        print(f"row = {row['order']} and expected_len = {expected_len}")
        return False

# Apply the validation function to the dataframe
comments_checkpoints_valid["is_valid_length"] = comments_checkpoints_valid.apply(validate_order_length, axis=1)

# Output the updated checkpoints_valid DataFrame
print(comments_checkpoints_valid)


Error processing row: 1gkbmkz
row = None and expected_len = 10
Error processing row: 1gi378c
row = None and expected_len = 15
        Type  Post_ID                                              Title  \
1    Comment  1gkukkp  AITA for slipping money into my boyfriend’s wa...   
2    Comment  1gkukkp  AITA for slipping money into my boyfriend’s wa...   
3    Comment  1gkukkp  AITA for slipping money into my boyfriend’s wa...   
4    Comment  1gkukkp  AITA for slipping money into my boyfriend’s wa...   
5    Comment  1gkukkp  AITA for slipping money into my boyfriend’s wa...   
..       ...      ...                                                ...   
809  Comment  1gi378c  AITA for yelling at my neighbour on the street...   
810  Comment  1gi378c  AITA for yelling at my neighbour on the street...   
811  Comment  1gi378c  AITA for yelling at my neighbour on the street...   
812  Comment  1gi378c  AITA for yelling at my neighbour on the street...   
813  Comment  1gi378c  AITA for yellin

/var/folders/rk/mnjxqksj6mb4svq2lvy3b5dw0000gn/T/ipykernel_26888/3482886931.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_checkpoints_valid["is_valid_length"] = comments_checkpoints_valid.apply(validate_order_length, axis=1)


In [30]:
len(comments_checkpoints_valid[comments_checkpoints_valid["is_valid_length"] == False])

50

In [31]:
filtered_comments_checkpoints_valid = comments_checkpoints_valid[comments_checkpoints_valid["is_valid_length"] == True]

# Analysis of the calculated ballots and their validity

In [32]:
# reading preferences from the lists

# aggregate by the post_id and concat all the orderings by a , to form an array
# {post : list of lists}


import pandas as pd

# Combine linear orderings into a new DataFrame with only Post_ID and combined rankings
def combine_orderings_to_new_df(df):
    # Select only Post_ID and Ranking
    reduced_df = df[["Post_ID", "order", "Type"]]  # Include the Type column for filtering
    filtered_df = reduced_df.loc[reduced_df["Type"] == "Comment"]  # Filter rows where Type is "Comment"
    
    # Group by Post_ID and combine rankings
    combined_df = (
        filtered_df.groupby("Post_ID")["order"]
        .apply(lambda x: list(x))  # Combine rankings into a list of lists
        .reset_index(name="preferences")
    )
    combined_df["preferences"] = combined_df["preferences"].apply(lambda x : np.array(x).T )
    return combined_df

# Call the function
post_preferences_df = combine_orderings_to_new_df(filtered_comments_checkpoints_valid)

post_preferences_df



,Post_ID,preferences
0,1gi2awo,"[[9, 6, 6, 1, 10, 9, 9, 9, 10, 9, 9, 9, 9, 6, ..."
1,1gi378c,"[[3, 2, 1, 3, 14, 3, 2, 3, 8, 3, 3, 1, 1, 3, 1..."
2,1gi5k6b,"[[14, 3, 14, 4, 3, 3, 3, 4, 3, 14, 3, 2, 2, 3,..."
3,1gi6rmt,"[[3, 2, 10, 2, 2, 10, 2, 2, 3, 3, 3, 3, 10, 3,..."
4,1giwkq5,"[[1, 1, 1, 2, 5, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2,..."
5,1giy3mj,"[[7, 8, 1, 2, 4, 5, 6, 6, 5, 7, 4, 7, 2, 8, 8,..."
6,1gj6zkq,"[[3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3,..."
7,1gjc9pm,"[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,..."
8,1gjquni,"[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,..."
9,1gjvwc2,"[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,..."


In [34]:
post_preferences_df.to_pickle('post_id_preferences.pkl')

In [16]:
preference_stats = {post_id : {} for post_id in post_preferences_df["Post_ID"]}
preference_stats

{'1gi2awo': {},
 '1gi378c': {},
 '1gi5k6b': {},
 '1gi6rmt': {},
 '1giwkq5': {},
 '1giy3mj': {},
 '1gj6zkq': {},
 '1gjc9pm': {},
 '1gjquni': {},
 '1gjvwc2': {},
 '1gjy9l8': {},
 '1gjyyhu': {},
 '1gk6tzm': {},
 '1gkbmkz': {},
 '1gkhfq7': {},
 '1gkhp7b': {},
 '1gkukkp': {}}

## Comparing the Hamming Distance

### Test Post ID

In [20]:
test = "1gi2awo"

preferences = post_preferences_df[post_preferences_df["Post_ID"] == test]["preferences"].values[0]
num_candidates = len(slates[test].dropna())
num_voters = preferences.shape[1]
test_op = hamming_distance_matrix(preferences)

# sanity check
assert test_op.shape == (num_voters,num_voters)

In [21]:
def hamming_distance_matrix(preference_matrix):
    # Convert to numpy array and transpose the matrix
    preference_matrix = np.array(preference_matrix).T
    n = preference_matrix.shape[0]  # Number of rows (original columns)
    distance_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                distance_matrix[i, j] = 0  # No distance with itself
            else:
                distance_matrix[i, j] = np.sum(preference_matrix[i] != preference_matrix[j])
    
    return distance_matrix


https://chatgpt.com/share/674bc0d8-7fa8-800e-acf5-e3ee90729e75

scroll to the the bottom

In [50]:
post_preferences_df

,Post_ID,preferences
0,1gi2awo,"[[9, 6, 6, 1, 10, 9, 9, 9, 10, 9, 9, 9, 9, 6, ..."
1,1gi378c,"[[3, 2, 1, 3, 14, 3, 2, 3, 8, 3, 3, 1, 1, 3, 1..."
2,1gi5k6b,"[[14, 3, 14, 4, 3, 3, 3, 4, 3, 14, 3, 2, 2, 3,..."
3,1gi6rmt,"[[3, 2, 10, 2, 2, 10, 2, 2, 3, 3, 3, 3, 10, 3,..."
4,1giwkq5,"[[1, 1, 1, 2, 5, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2,..."
5,1giy3mj,"[[7, 8, 1, 2, 4, 5, 6, 6, 5, 7, 4, 7, 2, 8, 8,..."
6,1gj6zkq,"[[3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3,..."
7,1gjc9pm,"[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,..."
8,1gjquni,"[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,..."
9,1gjvwc2,"[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,..."


In [51]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



def plot_hamming_distances_with_stats(distance_matrix, size):
    """Plot the heatmap of the Hamming distance matrix with summary statistics."""
    mean_distance = np.mean(distance_matrix)
    std_distance = np.std(distance_matrix)
    
    # Create a figure for the heatmap
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot the heatmap
    sns.heatmap(distance_matrix, annot=True, cmap='coolwarm', fmt='d', linewidths=0.5, ax=ax)
    
    # Add a text box with the summary statistics
    stats_text = f"Mean Hamming Distance: {mean_distance:.2f}\nStandard Deviation: {std_distance:.2f}"
    plt.gcf().text(0.85, 0.6, stats_text, fontsize=12, ha='center', va='center', 
                   bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=1'))
    
    # Set title and labels
    ax.set_title(f"Hamming Distance Matrix for Preference Vectors (Size {size})")
    ax.set_xlabel("Post Index")
    ax.set_ylabel("Post Index")
    
    plt.show()

# Filter the vectors by size 10, 12, and 15 and calculate the Hamming distance for each size
valid_sizes = [10, 12, 15]


for index, row in post_preferences_df.iterrows():
        # Proceed if there are vectors of this size
        if len(filtered_matrix) > 0:
            distance_matrix = hamming_distance_matrix(np.array(filtered_matrix))
            plot_hamming_distances_with_stats(distance_matrix, size)
        else:
            print(f"No vectors found with size {size}")


No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15
No vectors found with size 15


## Comparing Spearman's rank

In [23]:
from scipy.stats import spearmanr

def spearman_correlation_matrix(preference_matrix):
    n = len(preference_matrix)
    rho_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                rho_matrix[i, j] = 1  # Perfect correlation with itself
            else:
                rho, _ = spearmanr(preference_matrix[:, i], preference_matrix[:, j])  # Using columns after transposition
                rho_matrix[i, j] = rho
    return rho_matrix


In [24]:
test = "1gi2awo"
num_candidates = len(slates[test].dropna())
num_voters = preferences.shape[1]


preferences = post_preferences_df[post_preferences_df["Post_ID"] == test]["preferences"].values[0]

test_op = spearman_correlation_matrix(preferences)

test_op

array([[ 1.        , -0.43030303, -0.2969697 ,  0.05454545,  0.47878788,
         0.81818182,  0.80606061,  0.72121212, -0.04242424,  0.17575758],
       [-0.43030303,  1.        ,  0.5030303 ,  0.00606061, -0.38181818,
        -0.18787879, -0.04242424, -0.09090909,  0.35757576,  0.27272727],
       [-0.2969697 ,  0.5030303 ,  1.        ,  0.17575758, -0.04242424,
        -0.45454545, -0.39393939, -0.06666667,  0.76969697,  0.49090909],
       [ 0.05454545,  0.00606061,  0.17575758,  1.        , -0.01818182,
         0.01818182,  0.1030303 ,  0.30909091, -0.12727273,  0.33333333],
       [ 0.47878788, -0.38181818, -0.04242424, -0.01818182,  1.        ,
         0.2969697 ,  0.43030303,  0.66060606,  0.47878788,  0.21212121],
       [ 0.81818182, -0.18787879, -0.45454545,  0.01818182,  0.2969697 ,
         1.        ,  0.91515152,  0.75757576, -0.22424242,  0.16363636],
       [ 0.80606061, -0.04242424, -0.39393939,  0.1030303 ,  0.43030303,
         0.91515152,  1.        ,  0.86666667

https://chatgpt.com/share/674bc152-27a0-800e-9983-89a885efa3ee

## Comparing Kendall Tau Distance

In [25]:
# need to fix this! 

from scipy.stats import kendalltau

def kendall_tau_matrix(preference_matrix):
    # Assuming preference_matrix is now transposed
    n = len(preference_matrix)
    tau_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                tau_matrix[i, j] = 1  # Perfect agreement with itself
            else:
                tau, _ = kendalltau(preference_matrix[:, i], preference_matrix[:, j])
                tau_matrix[i, j] = tau
    return tau_matrix


In [26]:
test = "1gi2awo"
num_candidates = len(slates[test].dropna())
num_voters = preferences.shape[1]


preferences = post_preferences_df[post_preferences_df["Post_ID"] == test]["preferences"].values[0]

test_op = kendall_tau_matrix(preferences)

test_op

array([[ 1.        , -0.28888889, -0.2       ,  0.06666667,  0.37777778,
         0.68888889,  0.68888889,  0.64444444, -0.02222222,  0.15555556],
       [-0.28888889,  1.        ,  0.46666667,  0.02222222, -0.28888889,
        -0.15555556,  0.02222222, -0.11111111,  0.37777778,  0.28888889],
       [-0.2       ,  0.46666667,  1.        ,  0.11111111, -0.02222222,
        -0.33333333, -0.24444444, -0.02222222,  0.55555556,  0.46666667],
       [ 0.06666667,  0.02222222,  0.11111111,  1.        , -0.02222222,
         0.02222222,  0.11111111,  0.24444444, -0.06666667,  0.28888889],
       [ 0.37777778, -0.28888889, -0.02222222, -0.02222222,  1.        ,
         0.15555556,  0.33333333,  0.46666667,  0.33333333,  0.15555556],
       [ 0.68888889, -0.15555556, -0.33333333,  0.02222222,  0.15555556,
         1.        ,  0.82222222,  0.6       , -0.24444444,  0.11111111],
       [ 0.68888889,  0.02222222, -0.24444444,  0.11111111,  0.33333333,
         0.82222222,  1.        ,  0.68888889

In [29]:
for index, row in post_preferences_df.iterrows():
    idx = row["Post_ID"]
    pref  = row["preferences"]
    
    hamming_matrix = hamming_distance_matrix(pref)
    preference_stats[idx]["hamming"] = hamming_matrix

    spc = spearman_correlation_matrix(pref)
    preference_stats[idx]["spearman"] = spc

    kendalltau = hamming_distance_matrix(pref)
    preference_stats[idx]["kendalltau"] = kendalltau

In [36]:
# Saving the dictionary to a file
with open('preferences_distances.pkl', 'wb') as f:
    pickle.dump(preference_stats, f)

print("Data saved successfully.")

Data saved successfully.
